# 💥 Equilibrium Business Cycles: The Lucas Model

Why do economies experience business cycles, where output fluctuates around its potential? Robert Lucas's Nobel-winning work proposed a model where **unanticipated monetary shocks** can cause **real output fluctuations**, even in a world with rational agents and market clearing.

The key mechanism relies on **imperfect information**:
* Agents operate in different markets ("islands").
* They observe prices in their own market but cannot perfectly distinguish between changes in the *aggregate* price level (due to monetary factors) and changes in the *relative* price of their specific good (due to local factors).
* When an unexpected monetary injection raises all prices, some agents misinterpret the higher price in *their* market as a signal of higher relative demand for *their* good, leading them to increase production.

This simulation explores this core idea using a simplified framework with an Aggregate Demand (AD) curve and a **Lucas Aggregate Supply (AS) curve**.

# ⚙️ Simplified Model Setup (Log-Linear)

We use a simple log-linear model where variables represent log deviations from their baseline/potential levels:

1.  **Aggregate Demand (AD):** Derived from the quantity theory ($MV=PY$), assuming constant velocity and normalizing potential output $\bar{y}=0$.
    $$ y_t = m_t - p_t $$
    - $y_t$: Log output deviation from potential.
    - $m_t$: Log money supply (deviation from expected).
    - $p_t$: Log aggregate price level (deviation from expected).

2.  **Aggregate Supply (AS - Lucas Supply Curve):** Output deviates from potential ($\bar{y}=0$) only in response to *price surprises*.
    $$ y_t = \gamma (p_t - p_t^e) $$
    - $p_t^e$: Expected log price level based on information up to $t-1$.
    - $\gamma$: Sensitivity of output to price surprises. A higher $\gamma$ means output responds more strongly (agents are more easily "fooled" by aggregate price movements). This parameter reflects the underlying information structure (relative variances of aggregate vs. local shocks in the full model).

3.  **Expectations & Shocks:** We assume rational expectations. Agents expect money supply $m_t$ to be zero ($E_{t-1}m_t = 0$). Solving the model under this expectation yields $p_t^e = 0$. We then introduce a one-time, *unanticipated* monetary shock $\epsilon_m$, so the actual money supply becomes $m_t = \epsilon_m$.

4.  **Equilibrium:** Given the shock $m_t = \epsilon_m$ and the expectation $p_t^e = 0$, the equilibrium is found by solving the AD and AS equations simultaneously:
    $$ p_t = \frac{1}{1 + \gamma} \epsilon_m $$
    $$ y_t = \frac{\gamma}{1 + \gamma} \epsilon_m $$
    An unanticipated increase in money ($\epsilon_m > 0$) raises *both* prices and output.

# 📊 Interpreting the Simulation

The plot shows the AD-AS diagram in $(y, p)$ space (log deviations).

* **AS Curve:** $y = \gamma p$. Its slope is $1/\gamma$. A higher $\gamma$ (more confusion) makes the AS curve *flatter*.
* **AD Curve:** $y = m - p$. Its slope is -1. An unanticipated monetary shock $\epsilon_m$ shifts the AD curve *up/right* by the amount of the shock (since $m_t = \epsilon_m$).
* **Equilibrium:** The intersection of the AD and AS curves determines the resulting output ($y_t$) and price level ($p_t$) deviations.

**Key Parameter: $\gamma$**
* **High $\gamma$ (Flatter AS):** Agents are easily confused. A monetary shock leads to a *large output response* and a smaller price response. The economy deviates significantly from potential output.
* **Low $\gamma$ (Steeper AS):** Agents are good at distinguishing aggregate shocks. A monetary shock leads to a *small output response* and a larger price response. Monetary policy is closer to being neutral.

Adjust the sliders for the shock size ($\epsilon_m$) and the supply sensitivity ($\gamma$) to see how they affect the equilibrium outcome.

In [1]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Layout
from IPython.display import display, Markdown
import warnings

# Optional: Use a specific style
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except IOError:
    pass # Use default if style not found

def lucas_model_simulator(gamma=1.0, money_shock=2.0):
    """
    Simulates the equilibrium response to an unanticipated money shock
    in a simplified Lucas AD-AS model.

    Args:
        gamma (float): Lucas supply curve parameter (y = gamma * (p - p_e)). gamma > 0.
        money_shock (float): Unanticipated shock to log money supply (epsilon_m).
    """
    # Input validation
    gamma = max(gamma, 1e-6) # Ensure gamma is positive

    # --- Calculate Equilibrium ---
    # Expected price p_e = 0
    # Equilibrium price: p = (1 / (1 + gamma)) * money_shock
    p_eq = (1.0 / (1.0 + gamma)) * money_shock
    # Equilibrium output: y = gamma * p_eq = (gamma / (1 + gamma)) * money_shock
    y_eq = gamma * p_eq

    # --- Calculations for Plotting ---
    # Define range for output y for plotting curves
    y_max_abs = max(abs(y_eq) * 2, 1) # Ensure some range around equilibrium
    y_vals = np.linspace(-y_max_abs, y_max_abs, 200)

    # AS Curve: y = gamma * p => p = y / gamma
    p_as = y_vals / gamma

    # Original AD Curve (m=0): y = 0 - p => p = -y
    p_ad0 = -y_vals

    # Shifted AD Curve (m=money_shock): y = money_shock - p => p = money_shock - y
    p_ad1 = money_shock - y_vals

    # --- Plotting ---
    fig, ax = plt.subplots(figsize=(9, 7))

    # Plot AS Curve
    ax.plot(y_vals, p_as, label=f'AS Curve (y = {gamma:.2f}*p)', color='red', linewidth=2.5)

    # Plot Original AD Curve
    ax.plot(y_vals, p_ad0, label='AD Curve (Expected, m=0)', color='grey', linestyle='--', linewidth=2)

    # Plot Shifted AD Curve
    ax.plot(y_vals, p_ad1, label=f'AD Curve (Shock, m={money_shock:.2f})', color='blue', linewidth=2.5)

    # Plot Equilibrium Point
    ax.scatter(y_eq, p_eq, color='black', s=100, zorder=5, label=f'Equilibrium (y={y_eq:.2f}, p={p_eq:.2f})')
    # Annotate Equilibrium
    ax.annotate(f'Equilibrium\n y={y_eq:.2f}\n p={p_eq:.2f}', xy=(y_eq, p_eq),
                xytext=(20, -30) if y_eq < 0 else (-50, 30), textcoords='offset points',
                arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0.2'),
                fontsize=10, bbox=dict(boxstyle='round,pad=0.3', fc='white', alpha=0.7))

    # Add reference lines
    ax.axhline(0, color='grey', linestyle=':', linewidth=0.5)
    ax.axvline(0, color='grey', linestyle=':', linewidth=0.5) # y_bar = 0

    ax.set_title("Lucas Model: Response to Unanticipated Money Shock")
    ax.set_xlabel("Output (y, log deviation from potential)")
    ax.set_ylabel("Price Level (p, log deviation from expected)")
    ax.legend()
    ax.grid(True, linestyle='--', alpha=0.7)

    # Set plot limits dynamically
    p_range = max(abs(p_eq)*2, abs(money_shock)*1.1, 1)
    ax.set_xlim(-y_max_abs, y_max_abs)
    ax.set_ylim(-p_range, p_range)

    plt.tight_layout()
    plt.show()

    # --- Display Results ---
    results_md = f"""
    ### ⚙️ Equilibrium Results:

    * **Supply Sensitivity (gamma γ):** {gamma:.3f}
    * **Unanticipated Money Shock (epsilon_m):** {money_shock:.3f}
    * **Expected Price Level (p^e):** 0.000
    * ---
    * **Resulting Price Level (p):** **{p_eq:.4f}**
        * *Calculated as: $p = \\frac{{1}}{{1 + \\gamma}} \\epsilon_m$*
    * **Resulting Output (y):** **{y_eq:.4f}**
        * *Calculated as: $y = \\frac{{\\gamma}}{{1 + \\gamma}} \\epsilon_m$*

    *Interpretation: The shock $\\epsilon_m$ is split between prices and output. A higher $\\gamma$ means more of the shock affects output ($y$) and less affects prices ($p$).*
    """
    display(Markdown(results_md))


# --- Create Interactive Widgets ---
style = {'description_width': 'initial'}
layout = Layout(width='95%')

interact(
    lucas_model_simulator,
    gamma=FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1, description='Supply Sensitivity (gamma γ):', style=style, layout=layout, readout_format='.1f'),
    money_shock=FloatSlider(value=0.05, min=-0.1, max=0.1, step=0.01, description='Money Shock (epsilon_m):', style=style, layout=layout, readout_format='.3f')
);


interactive(children=(FloatSlider(value=1.0, description='Supply Sensitivity (gamma γ):', layout=Layout(width=…

In [2]:
# Lucas Islands Model of Business Cycles

# --- Imports ---
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider
from IPython.display import display, Markdown

# --- Model Function ---
def lucas_islands_model(std_agg_shock=0.05, std_local_shock=0.02, num_islands=50, periods=50):
    np.random.seed(0)

    # Simulate aggregate and local shocks
    agg_shocks = np.random.normal(0, std_agg_shock, periods)
    local_shocks = np.random.normal(0, std_local_shock, (num_islands, periods))

    # Actual prices = aggregate + local shocks
    prices = agg_shocks + local_shocks

    # Agents form expectations based only on their local price + average of all previous prices
    expected_agg = np.zeros(periods)
    output = np.zeros((num_islands, periods))

    a = 1.0  # Sensitivity parameter

    for t in range(1, periods):
        expected_agg[t] = np.mean(prices[:, t-1])  # Agents expect the last average price to persist
        for i in range(num_islands):
            perceived_rel_price = prices[i, t] - expected_agg[t]
            output[i, t] = a * perceived_rel_price  # Produce more if think price is high locally

    avg_output = np.mean(output, axis=0)

    # --- Plotting ---
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10,8))

    ax1.plot(np.arange(periods), agg_shocks.cumsum(), label='Cumulative Aggregate Shock', color='black')
    ax1.set_title('Aggregate Price Shocks Over Time')
    ax1.set_xlabel('Time Periods')
    ax1.set_ylabel('Price Level')
    ax1.grid(True, linestyle='--', alpha=0.6)
    ax1.legend()

    ax2.plot(np.arange(periods), avg_output, label='Aggregate Output Response', color='crimson')
    ax2.set_title('Average Output Across Islands')
    ax2.set_xlabel('Time Periods')
    ax2.set_ylabel('Output')
    ax2.axhline(0, color='grey', linestyle='--', linewidth=1)
    ax2.grid(True, linestyle='--', alpha=0.6)
    ax2.legend()

    plt.tight_layout()
    plt.show()

    # --- Markdown Summary ---
    summary = f"""
    ### 📘 Lucas Islands Model: Key Results

    - **Aggregate Shock Std Dev:** {std_agg_shock:.2f}
    - **Local Shock Std Dev:** {std_local_shock:.2f}
    - **Number of Markets:** {num_islands}

    > In periods with large aggregate shocks relative to local noise, output fluctuates significantly.
    > When local noise dominates, agents misinterpret signals less — leading to muted business cycles.
    """
    display(Markdown(summary))

# --- Widget for Interactivity ---
interact(lucas_islands_model,
         std_agg_shock=FloatSlider(value=0.05, min=0.01, max=0.2, step=0.01, description='Std Agg Shock'),
         std_local_shock=FloatSlider(value=0.02, min=0.0, max=0.1, step=0.005, description='Std Local Shock'),
         num_islands=IntSlider(value=50, min=10, max=200, step=10, description='Number of Islands'),
         periods=IntSlider(value=50, min=20, max=100, step=10, description='Periods'))


interactive(children=(FloatSlider(value=0.05, description='Std Agg Shock', max=0.2, min=0.01, step=0.01), Floa…

<function __main__.lucas_islands_model(std_agg_shock=0.05, std_local_shock=0.02, num_islands=50, periods=50)>